# Change of Chararacter CHOCH

### Load the data

In [111]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats


In [112]:
import yfinance as yf
import pandas as pd

# dataF = yf.download("EURJPY=X", start="2023-9-2", end="2023-10-2", interval='30m')

# get the data from last 60 days until the present
dataF = yf.download("EURJPY=X", period="60d", interval='60m')

dataF.iloc[:,:]
#dataF.Open.iloc

df = pd.DataFrame(dataF)

# use df index, convert DateTime to  a column instead of index
df.reset_index(inplace=True)

# delete Adj Close
df = df.drop(['Adj Close'], axis=1)

# rename Datetime to "Gmt time"

df = df.rename(columns={'Datetime':'Gmt time'})

df

[*********************100%***********************]  1 of 1 completed


,Gmt time,Open,High,Low,Close,Volume
0,2023-07-17 00:00:00,155.766998,155.802994,155.682007,155.703003,0
1,2023-07-17 01:00:00,155.703003,155.753998,155.326996,155.328995,0
2,2023-07-17 02:00:00,155.328003,155.559006,155.322998,155.427002,0
3,2023-07-17 03:00:00,155.421997,155.542007,155.369995,155.514999,0
4,2023-07-17 04:00:00,155.514999,155.697006,155.507996,155.669006,0
...,...,...,...,...,...,...
1425,2023-10-06 19:00:00,158.132996,158.259995,158.106995,158.203995,0
1426,2023-10-06 20:00:00,158.205002,158.235001,158.106995,158.141998,0
1427,2023-10-06 21:00:00,158.141006,158.145996,157.927002,158.078995,0
1428,2023-10-06 22:00:00,158.078995,158.078995,158.078995,158.078995,0


In [113]:

# df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
# df=df[df['volume']!=0]
# df.reset_index(drop=True, inplace=True)

df['RSI'] = ta.rsi(df.Close, length=12)
df['EMA'] = ta.ema(df.Close, length=150)
df.tail()

df=df[0:5000]

### Trend detection

In [114]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i], df.Close[i])>=df.EMA[i]:
            dnt=0
        if min(df.Open[i], df.Close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/3269106155.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [115]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [116]:
window=5
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/1837583093.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [117]:
def pointpos(x):
    if x['isPivot']==2:
        return x['Low']-1e-3
    elif x['isPivot']==1:
        return x['High']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/250927464.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [118]:
# dfpl = df[300:450]
dfpl = df[0:len(df)]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [119]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = localdf[localdf['isPivot'] == 1].High.tail(3).values
    idxhighs = localdf[localdf['isPivot'] == 1].High.tail(3).index
    lows = localdf[localdf['isPivot'] == 2].Low.tail(3).values
    idxlows = localdf[localdf['isPivot'] == 2].Low.tail(3).index

    pattern_detected = 0

    lim1 = 0.005
    lim2 = lim1/3
    if len(highs) == 3 and len(lows) == 3:
        order_condition = (idxlows[0] < idxhighs[0] 
                           < idxlows[1] < idxhighs[1] 
                           < idxlows[2] < idxhighs[2])
        order_condition = True
        diff_condition = ( 
                            abs(lows[0]-highs[0])>lim1 and 
                            abs(highs[0]-lows[1])>lim2 and
                            abs(highs[1]-lows[1])>lim1 and
                            abs(highs[1]-lows[2])>lim2
                            )
        diff_condition = True
        
        # this pattern means that the market is going to go up
        choch_up = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] > lows[1] and lows[2] < highs[1] and
            highs[2] < highs[1] and highs[2] > lows[2]
            )

        # this pattern means
        choch_down = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] < lows[1] and
            highs[2] < highs[1] 
            )

        if (order_condition and
            diff_condition and
            choch_up
        ):
            pattern_detected = 2

        if (order_condition and
            diff_condition and
            choch_down
        ):
            pattern_detected = 1

    return pattern_detected

In [120]:
df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=6))

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/2281627387.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [121]:
df[df['pattern_detected']!=0]

,Gmt time,Open,High,Low,Close,Volume,RSI,EMA,EMASignal,isPivot,pointpos,pattern_detected
80,2023-07-20 08:00:00,156.408005,156.537003,156.250000,156.453995,0,53.882414,NaN,3,1,156.538003,2
81,2023-07-20 09:00:00,156.455994,156.522003,156.130005,156.307999,0,47.767702,NaN,3,2,156.129005,2
172,2023-07-26 05:00:00,155.776001,155.927002,155.768997,155.824005,0,41.706933,156.249411,1,0,NaN,1
173,2023-07-26 06:00:00,155.820007,155.983994,155.798996,155.921997,0,47.952110,156.245074,1,1,155.984994,1
174,2023-07-26 07:00:00,155.923004,155.931000,155.574005,155.647995,0,36.141218,156.237166,1,0,NaN,1
175,2023-07-26 08:00:00,155.660004,155.830002,155.584000,155.701004,0,39.296743,156.230065,1,0,NaN,1
176,2023-07-26 09:00:00,155.695007,155.740005,155.253998,155.429993,0,30.806510,156.219468,1,0,NaN,1
177,2023-07-26 10:00:00,155.427994,155.438004,155.313004,155.403000,0,30.099904,156.208653,1,0,NaN,1
178,2023-07-26 11:00:00,155.403000,155.634003,155.343994,155.572006,0,39.567673,156.200221,1,0,NaN,1
179,2023-07-26 12:00:00,155.578003,155.585999,155.139999,155.289993,0,31.741465,156.188165,1,2,155.138999,1


In [122]:
def TotalSignal(l):
    # if (df.VWAPSignal[l]==2
    #     and df.Close[l]<=df['BBL.0'][l]
    #     and df.RSI[l]<45):
    #         return 2
    # if (df.VWAPSignal[l]==1
    #     and df.Close[l]>=df['BBU.0'][l]
    #     and df.RSI[l]>55):
    #         return 1
    # return 0

    if (
        df['pattern_detected'][l]==2
    ):
            return 2
    if (
        df['pattern_detected'][l]==1
    ):
            return 1
    return 0
        
TotSignal = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/3114384813.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [123]:
df[df.TotalSignal!=0].count()

Gmt time            38
Open                38
High                38
Low                 38
Close               38
Volume              38
RSI                 38
EMA                 36
EMASignal           38
isPivot             38
pointpos             7
pattern_detected    38
TotalSignal         38
dtype: int64

In [124]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/4045792666.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [125]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=0
dfpl = df[st:st+ len(df)]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                # go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                #            line=dict(color='blue', width=1), 
                #            name="VWAP"), 
                # go.Scatter(x=dfpl.index, y=dfpl['BBL.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBL"),
                # go.Scatter(x=dfpl.index, y=dfpl['BBU.0'], 
                #            line=dict(color='green', width=1), 
                #            name="BBU")
                # go.Scatter(x=dfpl.index, y=dfpl['100_MA'],
                #             line=dict(color='red', width=1), 
                #             name="100_MA"),
      ])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")

# set figure size
fig.update_layout(
    autosize=False,
    width=1600,
    height=1000,
    margin=dict(l=50,r=50,b=100,t=100,pad=4)
)

fig.show()

In [126]:
dfpl = df[0:len(df)].copy()
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [127]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.05
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.5*self.data.ATR[-1]
        TPSLRatio = 2.00

        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
                self.trades[-1].close()
        
        print("Datetime: ", self.data.index[-1])
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            print("Buy", self.data.Close[-1])
            print("Stop loss: ", sl1)
            print("Take profit: ", tp1)
            print("------")
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            print("Sell", self.data.Close[-1])
            print("Stop loss: ", sl1)
            print("Take profit: ", tp1)
            print("------")

        else:
            print("No signal")

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/100, commission=0.00)
stat = bt.run()

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/1255501625.py:44: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_80067/1255501625.py:44: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Datetime:  1
No signal
Datetime:  2
No signal
Datetime:  3
No signal
Datetime:  4
No signal
Datetime:  5
No signal
Datetime:  6
No signal
Datetime:  7
No signal
Datetime:  8
No signal
Datetime:  9
No signal
Datetime:  10
No signal
Datetime:  11
No signal
Datetime:  12
No signal
Datetime:  13
No signal
Datetime:  14
No signal
Datetime:  15
No signal
Datetime:  16
No signal
Datetime:  17
No signal
Datetime:  18
No signal
Datetime:  19
No signal
Datetime:  20
No signal
Datetime:  21
No signal
Datetime:  22
No signal
Datetime:  23
No signal
Datetime:  24
No signal
Datetime:  25
No signal
Datetime:  26
No signal
Datetime:  27
No signal
Datetime:  28
No signal
Datetime:  29
No signal
Datetime:  30
No signal
Datetime:  31
No signal
Datetime:  32
No signal
Datetime:  33
No signal
Datetime:  34
No signal
Datetime:  35
No signal
Datetime:  36
No signal
Datetime:  37
No signal
Datetime:  38
No signal
Datetime:  39
No signal
Datetime:  40
No signal
Datetime:  41
No signal
Datetime:  42
No signal
D

In [128]:
print(stat)

Start                                     0.0
End                                    1429.0
Duration                               1429.0
Exposure Time [%]                    10.48951
Equity Final [$]                    99.418505
Equity Peak [$]                    101.558673
Return [%]                          -0.581495
Buy & Hold Return [%]                1.455976
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -5.040145
Avg. Drawdown [%]                   -2.078701
Max. Drawdown Duration                 1235.0
Avg. Drawdown Duration                  336.5
# Trades                                 12.0
Win Rate [%]                        33.333333
Best Trade [%]                        0.43739
Worst Trade [%]                     -0.235552
Avg. Trade [%]                    

In [129]:
bt.plot(show_legend=False)

Row(id='3799', ...)